In [ ]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Array Data Types

In BigQuery, an [ARRAY](https://cloud.google.com/bigquery/docs/reference/standard-sql/data-types#array_type) (also called a `repeated` column) is an ordered list of zero or more elements of the same, non-`NULL` data type. It's important to note that BigQuery `ARRAY`s cannot contain nested `ARRAY`s. BigQuery DataFrames represents BigQuery `ARRAY` types to `pandas.ArrowDtype(pa.list_(T))`, where `T` is the underlying Arrow type of the array elements.

This notebook illustrates how to work with `ARRAY` columns in BigQuery DataFrames. First, let's import the required packages and perform the necessary setup below.

In [2]:
import bigframes.pandas as bpd
import bigframes.bigquery as bbq
import pandas as pd
import pyarrow as pa

In [ ]:
REGION = "US"  # @param {type: "string"}

bpd.options.display.progress_bar = None
bpd.options.bigquery.location = REGION

## Create DataFrames with array columns

**Example 1: Creating from a list of lists/tuples**

In [4]:
names = ["Alice", "Bob", "Charlie"]
scores = [
    [95, 88, 92],
    [78, 81],
    [82, 89, 94, 100]
]
df = bpd.DataFrame({"Name": names, "Scores": scores})
df

,Name,Scores
0,Alice,[95 88 92]
1,Bob,[78 81]
2,Charlie,[ 82 89 94 100]


In [5]:
df.dtypes

Name                 string[pyarrow]
Scores    list<item: int64>[pyarrow]
dtype: object

**Example 2: Defining schema explicitly**

In [6]:
bpd.Series(data=scores, dtype=pd.ArrowDtype(pa.list_(pa.float64())))

0            [95. 88. 92.]
1                [78. 81.]
2    [ 82.  89.  94. 100.]
dtype: list<item: double>[pyarrow]

**Example 3: Reading from a source**

In [7]:
bpd.read_gbq("bigquery-public-data.ml_datasets.credit_card_default", max_results=5)["predicted_default_payment_next_month"]

0    [{'tables': {'score': 0.9349926710128784, 'val...
1    [{'tables': {'score': 0.9690881371498108, 'val...
2    [{'tables': {'score': 0.8667634129524231, 'val...
3    [{'tables': {'score': 0.9351968765258789, 'val...
4    [{'tables': {'score': 0.8572560548782349, 'val...
Name: predicted_default_payment_next_month, dtype: list<item: struct<tables: struct<score: double, value: string>>>[pyarrow]

## Operate on `ARRAY` data

BigQuery DataFrames provides two main approaches for operating on list (`ARRAY`) data:

1. **The `Series.list` accessor**: Provides Pandas-like methods for array column manipulation.
2. **[BigQuery built-in functions](https://cloud.google.com/bigquery/docs/reference/standard-sql/array_functions)**: Allows you to use functions mirroring BigQuery SQL operations, available through the `bigframes.bigquery` module (abbreviated as `bbq` below), such as [`array_agg`](https://cloud.google.com/python/docs/reference/bigframes/latest/bigframes.bigquery#bigframes_bigquery_array_agg) and [`array_length`](https://cloud.google.com/python/docs/reference/bigframes/latest/bigframes.bigquery#bigframes_bigquery_array_length).

### Get the Length of Each Arrray

**Example 1: Using list accessor to get array length**

In [8]:
df['Scores'].list.len()

0    3
1    2
2    4
Name: Scores, dtype: Int64

**Example 2: Using BigQuery build-in functions to get array length**

In [9]:
bbq.array_length(df['Scores'])

0    3
1    2
2    4
Name: Scores, dtype: Int64

### Access Element at a Specific Index (e.g., First Element) 

In [10]:
df['Scores'].list[0]

0    95
1    78
2    82
Name: Scores, dtype: Int64

### Explode/Unnest Array elements into Seperate Rows

The exploded rows preserving original order when in ordering mode. If an array has multiple elements, exploded rows are ordered by the element's index
within its original array. 

In [11]:
scores = df['Scores'].explode()
scores

0     95
0     88
0     92
1     78
1     81
2     82
2     89
2     94
2    100
Name: Scores, dtype: Int64

### Aggregate elements back into an array

In [12]:
new_scores = scores + 5.0
new_scores_arr = bbq.array_agg(new_scores.groupby(level=0))
new_scores_arr

0         [100.  93.  97.]
1                [83. 86.]
2    [ 87.  94.  99. 105.]
Name: Scores, dtype: list<item: double>[pyarrow]

In [14]:
# Add adjusted scores into the DataFrame. This operation requires an implicit join 
# between the two tables, necessitating a unique index in the DataFrame (guaranteed 
# in the default ordering and index mode).
df['NewScores'] = new_scores_arr
df

,Name,Scores,NewScores
0,Alice,[95 88 92],[100. 93. 97.]
1,Bob,[78 81],[83. 86.]
2,Charlie,[ 82 89 94 100],[ 87. 94. 99. 105.]


# Struct Data Types

In BigQuery, a [struct](https://cloud.google.com/bigquery/docs/reference/standard-sql/data-types#struct_type) (also known as a `record`) is a collection of ordered fields, each with a defined data type (required) and an optional field name. BigQuery DataFrames maps BigQuery struct types to the pandas equivalent, `pandas.ArrowDtype(pa.struct())`. This section provides practical code examples illustrating how to use struct columns with BigQuery DataFrames.

## Create DataFrames with struct columns 

Create a DataFrame with an `Address` struct column by using dictionaries for the data and setting the dtype to `struct<City: string, State: string>[pyarrow]`.

In [13]:
names = bpd.Series(['Alice', 'Bob', 'Charlie'])
address = bpd.Series(
    [
        {'City': 'New York', 'State': 'NY'},
        {'City': 'San Francisco', 'State': 'CA'},
        {'City': 'Seattle', 'State': 'WA'}
    ],
    dtype=bpd.ArrowDtype(pa.struct(
         [('City', pa.string()), ('State', pa.string())]
    )))

df = bpd.DataFrame({'Name': names, 'Address': address})
df

/usr/local/google/home/sycai/src/python-bigquery-dataframes/venv/lib/python3.11/site-packages/google/cloud/bigquery/_pandas_helpers.py:570: UserWarning: Pyarrow could not determine the type of columns: bigframes_unnamed_index.
  warnings.warn(


,Name,Address
0,Alice,"{'City': 'New York', 'State': 'NY'}"
1,Bob,"{'City': 'San Francisco', 'State': 'CA'}"
2,Charlie,"{'City': 'Seattle', 'State': 'WA'}"


In [14]:
df.dtypes

Name                                    string[pyarrow]
Address    struct<City: string, State: string>[pyarrow]
dtype: object

## Operate on struct data

Similar to pandas, BigQuery DataFrames provides a [`StructAccessor`](https://cloud.google.com/python/docs/reference/bigframes/latest/bigframes.operations.structs.StructAccessor). Use the methods provided in this accessor to manipulate struct data.

In [15]:
# Return the dtype object of each child field of the struct.
df['Address'].struct.dtypes()

City     string[pyarrow]
State    string[pyarrow]
dtype: object

In [16]:
# Extract a child field as a Series
city = df['Address'].struct.field("City")
city

0         New York
1    San Francisco
2          Seattle
Name: City, dtype: string

In [17]:
# Extract all child fields of a struct as a DataFrame.
address_df = df['Address'].struct.explode()
address_df

,City,State
0,New York,NY
1,San Francisco,CA
2,Seattle,WA
